# Acoustic Data


In [53]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
import pandas as pd
from torchvision.io import read_image

In [54]:
TEST_CSV = "/run/media/thomas/Elements/Preprocessed/train/metadata_updated.csv"

In [55]:
from one_stage.dataset import DeeperShip
from dataset import create_data_loader
from preprocessing import (
    define_mel_spectrogram,
    define_gamma_spectrogram,
    define_cqt_spectrogram,
)

In [56]:
sample_rate = 32000
number_of_samples = 1
mel = define_gamma_spectrogram(sample_rate)

train_dataset = DeeperShip(
    TEST_CSV,
    sample_rate,
    number_of_samples,
    transform=mel,
)

In [57]:
type(train_dataset)

one_stage.dataset.DeeperShip

In [58]:
train_dataloader = create_data_loader(train_dataset, batch_size=25)
print(type(train_dataloader))

1
<class 'torch.utils.data.dataloader.DataLoader'>


In [59]:
from one_stage.model import ResNet18
from torch import nn

model = ResNet18(3, 5, 3)


def initialize_weights(model):
    if isinstance(model, nn.Linear) or isinstance(model, nn.Conv2d):
        if next(model.parameters()).is_cuda:  # Check if the model is on GPU
            device = next(model.parameters()).device  # Get the device of the model
            if isinstance(model, nn.Linear):
                nn.init.xavier_uniform_(model.weight)
                if model.bias is not None:
                    nn.init.zeros_(model.bias)
            elif isinstance(model, nn.Conv2d):
                nn.init.xavier_uniform_(model.weight)
                if model.bias is not None:
                    nn.init.zeros_(model.bias)
        else:
            if isinstance(model, nn.Linear):
                nn.init.xavier_uniform_(model.weight)
                if model.bias is not None:
                    nn.init.zeros_(model.bias)
            elif isinstance(model, nn.Conv2d):
                nn.init.xavier_uniform_(model.weight)
                if model.bias is not None:
                    nn.init.zeros_(model.bias)


model = model.cuda()
model.apply(initialize_weights)

ResNet18(
  (layer0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
  )
  (layer1): Sequential(
    (0): ResBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (shortcut): Sequential()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (shortcut): Sequential()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affin

In [60]:
from torchsummary import summary

print("Model Architecture")
print(summary(model, (3, 95, 126)))

Model Architecture
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 48, 63]           9,472
         MaxPool2d-2           [-1, 64, 24, 32]               0
       BatchNorm2d-3           [-1, 64, 24, 32]             128
              ReLU-4           [-1, 64, 24, 32]               0
            Conv2d-5           [-1, 64, 24, 32]          36,928
       BatchNorm2d-6           [-1, 64, 24, 32]             128
            Conv2d-7           [-1, 64, 24, 32]          36,928
       BatchNorm2d-8           [-1, 64, 24, 32]             128
          ResBlock-9           [-1, 64, 24, 32]               0
           Conv2d-10           [-1, 64, 24, 32]          36,928
      BatchNorm2d-11           [-1, 64, 24, 32]             128
           Conv2d-12           [-1, 64, 24, 32]          36,928
      BatchNorm2d-13           [-1, 64, 24, 32]             128
         ResBlock-14

In [61]:
learning_rate = 0.001
lr_schd_gamma = 0.9

loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimiser, gamma=lr_schd_gamma)

In [62]:
from tools.checkpoint import CheckpointManager, Checkpoint
from tools.train_manager import TrainManager
from torchmetrics import Accuracy, Precision, Recall, ConfusionMatrix, F1Score

In [64]:
epochs = 10
for epoch in range(epochs):
    for batch_idx, (inputs, targets) in enumerate(train_dataloader):
        print("test")
    #     optimiser.zero_grad()
    #     outputs = model(inputs)
    #     loss = loss_fn(outputs.squeeze(), targets)
    #     loss.backward()
    #     optimiser.step()

    #     if batch_idx % 10 == 0:
    #         print(
    #             "Epoch [{}/{}], Batch [{}/{}], Loss: {:.4f}".format(
    #                 epoch + 1, epochs, batch_idx + 1, len(train_dataloader), loss.item()
    #             )
    #         )

    # lr_scheduler.step()

AssertionError: Signal length shorter than reflect padding length (n_fft // 2).